# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,swakopmund,-22.6833,14.5333,63.45,88,21,6.38,NaN,1706402806
1,1,nouadhibou,20.9310,-17.0347,67.98,42,21,11.50,MR,1706402806
2,2,gazli,40.1333,63.4500,44.44,68,100,12.77,UZ,1706402807
3,3,st. john's,47.5649,-52.7093,20.97,87,0,8.05,CA,1706402807
4,4,margaret river,-33.9500,115.0667,67.35,83,100,12.82,AU,1706402807


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City"   
)

# Display the map plot
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

# Wind speed <= 5 m/s.
ideal_city_data_df = city_data_df[city_data_df["Wind Speed"] < 5]

# Zero cloudiness.
ideal_city_data_df = ideal_city_data_df[ideal_city_data_df["Cloudiness"] <= 10]

# 68 degrees F <= Max Temp <= 80 degrees F
ideal_city_data_df = ideal_city_data_df[(ideal_city_data_df["Max Temp"] < 80) & (ideal_city_data_df["Max Temp"] > 67)]

# Drop any rows with null values
ideal_city_data_df = ideal_city_data_df.dropna()


# Display sample data
ideal_city_data_df.head(30)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
132,132,ciudad lazaro cardenas,17.9583,-102.2000,77.07,76,0,4.94,MX,1706402837
138,138,brisas de zicatela,15.8369,-97.0419,78.49,69,0,2.93,MX,1706402838
155,155,coahuayana de hidalgo,18.7000,-103.6583,73.31,84,0,2.33,MX,1706402842
167,167,lazaro cardenas,17.9583,-102.2000,77.07,76,0,4.94,MX,1706402837
272,272,san luis de la loma,17.2706,-100.8939,76.91,75,0,4.79,MX,1706402869
275,275,copala,16.6500,-98.9833,77.49,74,0,3.36,MX,1706402870
387,387,port saint john's,-31.6229,29.5448,72.19,84,10,4.16,ZA,1706402896
480,480,cabo san lucas,22.8909,-109.9124,79.90,58,0,4.00,MX,1706402917
486,486,axim,4.8699,-2.2405,78.08,84,0,2.84,GH,1706402918
500,500,ciudad de armeria,18.9333,-103.9667,72.72,81,0,2.98,MX,1706402921


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_data_df.loc[:, ["City","Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
132,ciudad lazaro cardenas,MX,17.9583,-102.2000,76,
138,brisas de zicatela,MX,15.8369,-97.0419,69,
155,coahuayana de hidalgo,MX,18.7000,-103.6583,84,
167,lazaro cardenas,MX,17.9583,-102.2000,76,
272,san luis de la loma,MX,17.2706,-100.8939,75,
275,copala,MX,16.6500,-98.9833,74,
387,port saint john's,ZA,-31.6229,29.5448,84,
480,cabo san lucas,MX,22.8909,-109.9124,58,
486,axim,GH,4.8699,-2.2405,84,
500,ciudad de armeria,MX,18.9333,-103.9667,81,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
limit = 20

# set up a parameters dictionary
params = {
    "categories": categories,
    "limit": limit,
    "apiKey": geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    #curr_lat = hotel_df.at[index,'Lat']
    curr_lat = row['Lat']
    #curr_lng = hotel_df.at[index,'Lng']
    curr_lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    filters = f'circle:{curr_lng},{curr_lat},{radius}'
    bias = f"proximity:{curr_lng},{curr_lat}"
    
    params["filter"] = filters
    params["bias"] = bias
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    # Convert the API response to JSON format
    name_address = name_address.json()
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ciudad lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
brisas de zicatela - nearest hotel: Casa de Olas
coahuayana de hidalgo - nearest hotel: No hotel found
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
san luis de la loma - nearest hotel: No hotel found
copala - nearest hotel: No hotel found
port saint john's - nearest hotel: Outback Inn
cabo san lucas - nearest hotel: Comfort Rooms
axim - nearest hotel: Axim Beach Hotel
ciudad de armeria - nearest hotel: Hotel La Herradura
nobres - nearest hotel: No hotel found
vallenar - nearest hotel: Hotel del Marqués
itaberai - nearest hotel: Ell Palace Hotel
todos santos - nearest hotel: Hotel Casa Tota


,City,Country,Lat,Lng,Humidity,Hotel Name
132,ciudad lazaro cardenas,MX,17.9583,-102.2000,76,Hotel Sol del Pacífico
138,brisas de zicatela,MX,15.8369,-97.0419,69,Casa de Olas
155,coahuayana de hidalgo,MX,18.7000,-103.6583,84,No hotel found
167,lazaro cardenas,MX,17.9583,-102.2000,76,Hotel Sol del Pacífico
272,san luis de la loma,MX,17.2706,-100.8939,75,No hotel found
275,copala,MX,16.6500,-98.9833,74,No hotel found
387,port saint john's,ZA,-31.6229,29.5448,84,Outback Inn
480,cabo san lucas,MX,22.8909,-109.9124,58,Comfort Rooms
486,axim,GH,4.8699,-2.2405,84,Axim Beach Hotel
500,ciudad de armeria,MX,18.9333,-103.9667,81,Hotel La Herradura


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
humid_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City",
    hover_cols=['Country', 'Hotel Name']
)

# Display the map
humid_map


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)